In [ ]:
import random
import numpy as np
import json
import os
from shapely.geometry import Polygon, MultiPolygon, box
from shapely.affinity import rotate, translate
from shapely.ops import unary_union
from ezdxf import new
from ezdxf.math import Vec3
import matplotlib.pyplot as plt

# Устанавливаем глобальную точность для numpy
np.set_printoptions(precision=6, suppress=True)
NP_FLOAT_DTYPE = np.float32  # Используем float32 для единообразия

class Config:
    """Конфигурация генератора участков и зданий."""
    def __init__(self):
        self.num_polygons = 100
        self.num_variants = 100
        self.min_area = 3000
        self.max_attempts = 10000
        self.coord_range = (0, 80)
        self.image_size = 3000
        self.max_points = 8
        self.max_buildings = 1
        self.building_color = "#FFFF00"
        self.rotation_steps = list(range(0, 360, 10))
        self.min_distance = 0
        self.regular_building_prob = 0.2  # Вероятность обычных зданий
        self.l_building_prob = 0.4       # Вероятность Г-образных зданий
        self.combined_building_prob = 0.4 # Вероятность комбинированных зданий
        self.skip_center_percent = 30
        self.min_floors = 2
        self.max_floors = 25
        
        self.building_types = [
            (26, 16), (28, 16), (26, 18), 
            (18, 18), (16, 26), (16, 28), (18, 26)
        ]
        
        self.combined_buildings = [
            (54, 16), (44, 18), (26, 34)
        ]
        
        self.output_dirs = {
            "dxf": "./generated_db/dxf",
            "json": "./generated_db/json",
            "png": "./generated_db/png",
            "numpy": "./generated_db/numpy"
        }
        
        # Таблица максимальной плотности жилой застройки (тыс. кв.м/га)
        self.max_density_table = {
            3: 10.0,
            4: 11.8,
            5: 13.3,
            6: 14.5,
            7: 15.5,
            8: 16.4,
            9: 17.1,
            10: 17.8,
            11: 18.3,
            12: 18.8,
            13: 19.2,
            14: 19.6,
            15: 20.0,
            16: 20.3,
            17: 20.6,
            18: 20.9,
            19: 21.1,
            20: 21.3,
            21: 21.5,
            22: 21.7,
            23: 21.9,
            24: 22.1,
            25: 22.2
        }

        # Параметры отступов от границы участка
        self.setback_rules = {
            (2, 4): 8,    # Для этажности от 2 до 4 - отступ 8 м
            (5, 8): 8,    # Для этажности от 5 до 8 - отступ 8 м
            (9, float('inf')): 12  # Для этажности 9 и более - отступ 12 м
        }


class BuildingGenerator:
    """Генератор зданий различных типов."""
    def __init__(self, config):
        self.config = config
        self.l_building_types = self._generate_l_building_types()
        
    def _generate_l_building_types(self):
        """Генерирует возможные Г-образные комбинации зданий."""
        forbidden_pairs = {(16, 18), (26, 28), (18, 16),
        (28, 26),(18, 18), (16, 16), (26, 26)}
        l_buildings = set()
        
        for i, (w1, h1) in enumerate(self.config.building_types):
            for j, (w2, h2) in enumerate(self.config.building_types):
                if i != j and (w1, w2) not in forbidden_pairs:
                    l_buildings.add((w1, h1, w2, h2))
        
        return list(l_buildings)
    
    def generate_rotated_building(self, width, length, x, y, angle):
        """Генерирует прямоугольное здание с поворотом."""
        building = box(x - width/2, y - length/2, x + width/2, y + length/2)
        rotated = rotate(building, angle, origin='centroid', use_radians=False)
        return rotated
    
    def generate_l_shaped_building(self, width1, height1, width2, height2, angle=0):
        """Генерирует Г-образное здание."""
        rect1 = box(0, 0, width1, height1)
        rect2 = box(width1 - width2, height1, width1, height1 + height2)
        l_shape = unary_union([rect1, rect2])
        
        if angle != 0:
            l_shape = rotate(l_shape, angle, origin='centroid', use_radians=False)
        
        return l_shape


class PlotGenerator:
    """Генератор участков и их визуализации."""
    def __init__(self, config):
        self.config = config
        self.building_gen = BuildingGenerator(config)
        
    def generate_valid_polygon(self, skip_center_percent=30):
        """Генерирует валидный полигон участка с целочисленными координатами."""
        min_x, max_x = self.config.coord_range
        range_x = max_x - min_x
        skip_width = range_x * (skip_center_percent / 100)
        left_zone_max = min_x + (range_x - skip_width) / 2
        right_zone_min = max_x - (range_x - skip_width) / 2
        
        points = []
        for _ in range(random.randint(4, self.config.max_points)):
            x = random.randint(min_x, int(left_zone_max)) if random.random() < 0.5 else random.randint(int(right_zone_min), max_x)
            y = random.randint(min_x, int(left_zone_max)) if random.random() < 0.5 else random.randint(int(right_zone_min), max_x)
            points.append([x, y])
        
        center = np.mean(points, axis=0)
        angles = np.arctan2(np.array(points)[:,1] - center[1], np.array(points)[:,0] - center[0])
        points = np.array(points)[np.argsort(angles)]
        
        polygon = Polygon(points)
        if not polygon.is_valid or polygon.area < self.config.min_area:
            return self.generate_valid_polygon(skip_center_percent)
        
        return points.astype(NP_FLOAT_DTYPE)  # Приводим к заданному типу
    
    def can_place_building(self, building, buildings, min_distance):
        """Проверяет возможность размещения здания."""
        new_poly = Polygon(building["points"])
        for existing in buildings:
            if new_poly.distance(Polygon(existing["points"])) < min_distance:
                return False
        return True
    
    def generate_attached_building(self, base_building, offset_poly_shapely):
        """Генерирует примыкающее здание."""
        base_poly = Polygon(base_building["points"])
        bounds = base_poly.bounds
        base_width = bounds[2] - bounds[0]
        base_length = bounds[3] - bounds[1]
        
        rand_val = random.random()
        
        # Обычное здание
        if rand_val < self.config.regular_building_prob:
            width, length = random.choice(self.config.building_types)
            angle = random.choice(self.config.rotation_steps)
            
            for side in random.sample(['top', 'right', 'bottom', 'left'], 4):
                x, y = self._get_side_coords(side, bounds, base_width, base_length, width/2, length/2)
                building = self.building_gen.generate_rotated_building(width, length, x, y, angle)
                
                if offset_poly_shapely.contains(building):
                    building_area = width * length
                    return {
                        "points": np.array(building.exterior.coords, dtype=NP_FLOAT_DTYPE),
                        "type": (width, length),
                        "angle": angle,
                        "size_label": f"{width}x{length} м",
                        "area": float(building_area)
                    }
        
        # Г-образное здание
        elif rand_val < self.config.regular_building_prob + self.config.l_building_prob and self.building_gen.l_building_types:
            w1, h1, w2, h2 = random.choice(self.building_gen.l_building_types)
            angle = random.choice(self.config.rotation_steps)
            
            for side in random.sample(['top', 'right', 'bottom', 'left'], 4):
                x, y = self._get_side_coords(side, bounds, base_width, base_length, max(w1, w2)/2, max(h1, h2)/2)
                building = self.building_gen.generate_l_shaped_building(w1, h1, w2, h2, angle)
                building = translate(building, xoff=x, yoff=y)
                
                if offset_poly_shapely.contains(building):
                    building_area = w1 * h1 + w2 * h2
                    return {
                        "points": np.array(building.exterior.coords, dtype=NP_FLOAT_DTYPE),
                        "type": f"L-{w1}x{h1}+{w2}x{h2}",
                        "angle": angle,
                        "size_label": f"L-{w1}x{h1}+{w2}x{h2} м",
                        "area": float(building_area)
                    }
        
        # Комбинированное здание
        else:
            width, length = random.choice(self.config.combined_buildings)
            angle = random.choice(self.config.rotation_steps)
            
            for side in random.sample(['top', 'right', 'bottom', 'left'], 4):
                x, y = self._get_side_coords(side, bounds, base_width, base_length, width/2, length/2)
                building = self.building_gen.generate_rotated_building(width, length, x, y, angle)
                
                if offset_poly_shapely.contains(building):
                    building_area = width * length
                    return {
                        "points": np.array(building.exterior.coords, dtype=NP_FLOAT_DTYPE),
                        "type": (width, length),
                        "angle": angle,
                        "size_label": f"{width}x{length} м",
                        "area": float(building_area)
                    }
        
        return None
    
    def _get_side_coords(self, side, bounds, base_width, base_length, width_offset, length_offset):
        """Возвращает координаты для размещения здания относительно стороны."""
        if side == 'top':
            return bounds[0] + base_width/2, bounds[3] + length_offset
        elif side == 'right':
            return bounds[2] + width_offset, bounds[1] + base_length/2
        elif side == 'bottom':
            return bounds[0] + base_width/2, bounds[1] - length_offset
        else:  # left
            return bounds[0] - width_offset, bounds[1] + base_length/2
    
    def generate_buildings(self, offset_poly, max_buildings, max_footprint_area):
        """Генерирует здания внутри зоны отступов."""
        if offset_poly is None or len(offset_poly) < 3:
            return []
        
        offset_poly_shapely = Polygon(offset_poly)
        if offset_poly_shapely.area < self.config.min_area / 10:
            return []
        
        buildings = []
        building_shapes = []
        total_area = 0
        
        for _ in range(max_buildings):
            if buildings and random.random() < 0.7:
                new_building = self.generate_attached_building(random.choice(buildings), offset_poly_shapely)
                if new_building:
                    new_area = new_building["area"]
                    
                    if total_area + new_area <= max_footprint_area:
                        new_poly = Polygon(new_building["points"])
                        valid = True
                        for existing in building_shapes:
                            if new_poly.intersects(existing) or new_poly.distance(existing) < self.config.min_distance:
                                valid = False
                                break
                        
                        if valid:
                            buildings.append(new_building)
                            building_shapes.append(new_poly)
                            total_area += new_area
                            continue
            
            building = self._generate_random_building(offset_poly_shapely)
            if building and total_area + building["area"] <= max_footprint_area:
                if self.can_place_building(building, buildings, self.config.min_distance):
                    buildings.append(building)
                    building_shapes.append(Polygon(building["points"]))
                    total_area += building["area"]
        
        return buildings
    
    def generate_variants(self, poly, offset_poly, max_buildings):
        """Генерирует несколько вариантов чертежей с разной этажностью."""
        variants = []
        
        for _ in range(self.config.num_variants):
            floors = random.randint(self.config.min_floors, self.config.max_floors)
            stbck = self.get_setback(floors)
            
            offset_poly = self.create_offset_polygon(poly, stbck)
            if offset_poly is None:
                continue
                
            area = Polygon(poly).area
            _, max_footprint_area = self.calculate_max_construction_area(area, floors)
            
            buildings = self.generate_buildings(offset_poly, max_buildings, max_footprint_area)
            if buildings:
                variants.append({
                    "buildings": buildings,
                    "floors": floors,
                    "setback": stbck,
                    "offset_poly": offset_poly
                })
        
        return variants
    
    def _generate_random_building(self, offset_poly_shapely):
        """Генерирует случайное здание в пределах полигона."""
        rand_val = random.random()
        attempts = 100
        
        # Обычное здание
        if rand_val < self.config.regular_building_prob:
            width, length = random.choice(self.config.building_types)
            angle = random.choice(self.config.rotation_steps)
            
            for _ in range(attempts):
                coords = self._get_random_coords(offset_poly_shapely, width/2, length/2)
                if coords is None:
                    continue
                    
                x, y = coords
                building = self.building_gen.generate_rotated_building(width, length, x, y, angle)
                
                if offset_poly_shapely.contains(building):
                    building_area = width * length
                    return {
                        "points": np.array(building.exterior.coords, dtype=NP_FLOAT_DTYPE),
                        "type": (width, length),
                        "angle": angle,
                        "size_label": f"{width}x{length} м",
                        "area": float(building_area)
                    }
        
        # Г-образное здание
        elif rand_val < self.config.regular_building_prob + self.config.l_building_prob and self.building_gen.l_building_types:
            w1, h1, w2, h2 = random.choice(self.building_gen.l_building_types)
            angle = random.choice(self.config.rotation_steps)
            
            for _ in range(attempts):
                coords = self._get_random_coords(offset_poly_shapely, max(w1, w2)/2, max(h1, h2)/2)
                if coords is None:
                    continue
                    
                x, y = coords
                building = self.building_gen.generate_l_shaped_building(w1, h1, w2, h2, angle)
                building = translate(building, xoff=x, yoff=y)
                
                if offset_poly_shapely.contains(building):
                    building_area = w1 * h1 + w2 * h2
                    return {
                        "points": np.array(building.exterior.coords, dtype=NP_FLOAT_DTYPE),
                        "type": f"L-{w1}x{h1}+{w2}x{h2}",
                        "angle": angle,
                        "size_label": f"L-{w1}x{h1}+{w2}x{h2} м",
                        "area": float(building_area)
                    }
        
        # Комбинированное здание
        else:
            width, length = random.choice(self.config.combined_buildings)
            angle = random.choice(self.config.rotation_steps)
            
            for _ in range(attempts):
                coords = self._get_random_coords(offset_poly_shapely, width/2, length/2)
                if coords is None:
                    continue
                    
                x, y = coords
                building = self.building_gen.generate_rotated_building(width, length, x, y, angle)
                
                if offset_poly_shapely.contains(building):
                    building_area = width * length
                    return {
                        "points": np.array(building.exterior.coords, dtype=NP_FLOAT_DTYPE),
                        "type": (width, length),
                        "angle": angle,
                        "size_label": f"{width}x{length} м",
                        "area": float(building_area)
                    }
        
        return None
    
    def _get_random_coords(self, offset_poly_shapely, width_offset, length_offset):
        """Возвращает случайные координаты в пределах полигона."""
        minx, miny, maxx, maxy = offset_poly_shapely.bounds
        
        min_x = minx + width_offset
        max_x = maxx - width_offset
        min_y = miny + length_offset
        max_y = maxy - length_offset
        
        if min_x > max_x or min_y > max_y:
            center = offset_poly_shapely.centroid
            return center.x, center.y
        
        return random.uniform(min_x, max_x), random.uniform(min_y, max_y)
    
    def create_offset_polygon(self, points, setback):
        """Создает полигон с отступом."""
        polygon = Polygon(points)
        if setback <= 0:
            return None
        
        offset_poly = polygon.buffer(-setback, join_style=2)
        
        if offset_poly.is_empty:
            return None
        
        if isinstance(offset_poly, MultiPolygon):
            largest = max(offset_poly.geoms, key=lambda p: p.area)
            return np.array(largest.exterior.coords, dtype=NP_FLOAT_DTYPE)
        
        return np.array(offset_poly.exterior.coords, dtype=NP_FLOAT_DTYPE)
    
    def get_setback(self, floors):
        """Возвращает отступ в зависимости от этажности."""
        for (min_floor, max_floor), setback in self.config.setback_rules.items():
            if min_floor <= floors <= max_floor:
                return setback
        return 8
    
    def calculate_max_construction_area(self, plot_area, floors):
        """Вычисляет максимальную площадь застройки."""
        density = self.config.max_density_table.get(min(floors, 25), 22.2)
        plot_area_ha = plot_area / 10000
        max_living_area = density * plot_area_ha * 1000
        max_footprint_area = max_living_area / floors / 0.7
        
        return float(max_living_area), float(max_footprint_area)


class FileSaver:
    """Класс для сохранения результатов в различные форматы."""
    def __init__(self, config):
        self.config = config
    
    def save_as_dxf(self, points, offset_points, buildings, filename, floors, stbck):
        """Сохраняет в DXF файл."""
        doc = new(setup=True)
        msp = doc.modelspace()
        
        valid_points = points[~np.isnan(points).any(axis=1)]
        polygon = Polygon(valid_points)
        area = polygon.area
        
        max_living_area, max_footprint_area = self.calculate_max_construction_area(area, floors)
        footprint_area = sum(b["area"] for b in buildings)
        
        # Добавляем границы участка
        dxf_points = [Vec3(float(x), float(y)) for x, y in valid_points]
        dxf_points.append(dxf_points[0])
        msp.add_lwpolyline(dxf_points)
        
        # Добавляем зону отступа (если есть)
        if offset_points is not None:
            valid_offset_points = offset_points[~np.isnan(offset_points).any(axis=1)]
            if len(valid_offset_points) > 2:
                dxf_offset_points = [Vec3(float(x), float(y)) for x, y in valid_offset_points]
                dxf_offset_points.append(dxf_offset_points[0])
                msp.add_lwpolyline(dxf_offset_points, dxfattribs={'color': 140})
        
        # Добавляем здания
        for building in buildings:
            building_points = [Vec3(float(x), float(y)) for x, y in building["points"]]
            building_points.append(building_points[0])
            msp.add_lwpolyline(building_points, dxfattribs={'color': 2})
            
            center = Vec3(*np.mean(building["points"], axis=0))
            text = msp.add_text(
                building["size_label"],
                dxfattribs={'height': 2, 'insert': center, 'color': 2}
            )
            text.dxf.halign = 1
            text.dxf.valign = 1
        
        # Добавляем информацию об участке
        center = Vec3(*np.nanmean(points, axis=0))
        text = msp.add_text(
            f"Граница участка\nЭтажность: {floors}\nОтступ: {stbck}м\n"
            f"Площадь: {area:.1f} м²\n"
            f"Макс. площадь жилой застройки: {max_living_area:.1f} м²\n"
            f"Макс. площадь пятна застройки: {max_footprint_area:.1f} м²\n"
            f"Факт. площадь пятна застройки: {footprint_area:.1f} м²\n"
            f"Зданий: {len(buildings)}",
            dxfattribs={'height': 5, 'insert': center}
        )
        text.dxf.halign = 1
        text.dxf.valign = 1
        
        doc.saveas(filename)
    
    def save_as_json(self, points, offset_points, buildings, filename, floors, stbck):
        """Сохраняет в JSON файл."""
        valid_points = points[~np.isnan(points).any(axis=1)]
        polygon = Polygon(valid_points)
        area = polygon.area
        
        max_living_area, max_footprint_area = self.calculate_max_construction_area(area, floors)
        footprint_area = sum(b["area"] for b in buildings)
        
        data = {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [valid_points.tolist() + valid_points[0].tolist()],
                "area": float(area)
            },
            "properties": {
                "area": float(area),
                "num_points": len(valid_points),
                "offset": float(stbck),
                "floors": int(floors),
                "max_living_area": float(max_living_area),
                "max_footprint_area": float(max_footprint_area),
                "footprint_area": float(footprint_area),
                "buildings_count": len(buildings),
                "buildings": []
            }
        }
        
        if offset_points is not None:
            valid_offset_points = offset_points[~np.isnan(offset_points).any(axis=1)]
            if len(valid_offset_points) > 2:
                offset_polygon = Polygon(valid_offset_points)
                data["offset_geometry"] = {
                    "type": "Polygon",
                    "coordinates": [valid_offset_points.tolist() + valid_offset_points[0].tolist()],
                    "area": float(offset_polygon.area)
                }
                data["properties"]["offset_area"] = float(offset_polygon.area)
        
        for building in buildings:
            data["properties"]["buildings"].append({
                "type": building["type"],
                "angle": float(building["angle"]),
                "size_label": building["size_label"],
                "coordinates": building["points"].tolist(),
                "area": float(building["area"])
            })
        
        with open(filename, 'w') as f:
            json.dump(data, f, indent=2)
    
    def save_as_png(self, points, offset_points, buildings, filename, floors, stbck):
        """Создает PNG изображение."""
        valid_points = points[~np.isnan(points).any(axis=1)]
        polygon = Polygon(valid_points)
        area = polygon.area
        
        max_living_area, max_footprint_area = self.calculate_max_construction_area(area, floors)
        footprint_area = sum(b["area"] for b in buildings)
        
        fig, ax = plt.subplots(figsize=(30, 30), dpi=100)
        ax.set_xlim(0, self.config.image_size)
        ax.set_ylim(0, self.config.image_size)
        ax.axis('off')
        ax.set_facecolor('black')
        fig.patch.set_facecolor('black')
        
        scale_factor = self.config.image_size / self.config.coord_range[1]
        points_scaled = valid_points * scale_factor
        
        # Рисуем границы участка
        polygon = plt.Polygon(points_scaled, fill=False, edgecolor='white', linewidth=3)
        ax.add_patch(polygon)
        
        # Рисуем зону отступа (если есть)
        if offset_points is not None:
            valid_offset_points = offset_points[~np.isnan(offset_points).any(axis=1)]
            if len(valid_offset_points) > 2:
                offset_points_scaled = valid_offset_points * scale_factor
                offset_polygon = plt.Polygon(offset_points_scaled, fill=False, edgecolor='#00BFFF', linewidth=3)
                ax.add_patch(offset_polygon)
                offset_area = Polygon(valid_offset_points).area
                area_text = (f"Площадь участка: {area:.1f} м²\n"
                            f"Площадь с отступом: {offset_area:.1f} м²\n"
                            f"Макс. площадь жилой застройки: {max_living_area:.1f} м²\n"
                            f"Макс. площадь пятна застройки: {max_footprint_area:.1f} м²\n"
                            f"Факт. площадь пятна застройки: {footprint_area:.1f} м²")
            else:
                area_text = (f"Площадь участка: {area:.1f} м²\n"
                           f"Макс. площадь жилой застройки: {max_living_area:.1f} м²\n"
                           f"Макс. площадь пятна застройки: {max_footprint_area:.1f} м²\n"
                           f"Факт. площадь пятна застройки: {footprint_area:.1f} м²")
        else:
            area_text = (f"Площадь участка: {area:.1f} м²\n"
                       f"Макс. площадь жилой застройки: {max_living_area:.1f} м²\n"
                       f"Макс. площадь пятна застройки: {max_footprint_area:.1f} м²\n"
                       f"Факт. площадь пятна застройки: {footprint_area:.1f} м²")
        
        # Рисуем здания
        for building in buildings:
            building_points_scaled = building["points"] * scale_factor
            building_poly = plt.Polygon(building_points_scaled, fill=False, edgecolor=self.config.building_color, linewidth=2)
            ax.add_patch(building_poly)
            
            center = np.mean(building_points_scaled, axis=0)
            ax.text(center[0], center[1], 
                   building["size_label"], 
                   color='white', ha='center', va='center', fontsize=12)
        
        # Добавляем информацию об участке
        center = np.nanmean(points_scaled, axis=0)
        ax.text(center[0], center[1], 
               f"Этажность: {floors}\nОтступ: {stbck}м\n{area_text}\nЗданий: {len(buildings)}", 
               color='white', ha='center', va='center', fontsize=20)
        
        plt.savefig(filename, bbox_inches='tight', pad_inches=0)
        plt.close()
    
    def save_as_numpy(self, points, offset_points, buildings, filename, floors, stbck):
        """Сохраняет данные в формате numpy."""
        valid_points = points[~np.isnan(points).any(axis=1)]
        polygon = Polygon(valid_points)
        polygon_area = polygon.area
        
        max_living_area, max_footprint_area = self.calculate_max_construction_area(polygon_area, floors)
        footprint_area = sum(b["area"] for b in buildings)
        
        # Подготовка данных о зданиях
        buildings_data = []
        for b in buildings:
            building_points = b["points"].astype(NP_FLOAT_DTYPE)
            center = np.mean(building_points, axis=0)
            angle = b.get("angle", 0)
            building_type = b.get("type", "unknown")
            area = b.get("area", 0)
            
            buildings_data.append({
                "points": building_points,
                "center": center.astype(NP_FLOAT_DTYPE),  # Координаты середины здания
                "angle": float(angle),                     # Угол поворота здания
                "type": str(building_type),               # Тип/форма здания
                "area": float(area)                        # Площадь здания
            })
        
        data = {
            "plot_points": valid_points.astype(NP_FLOAT_DTYPE),
            "offset_points": offset_points[~np.isnan(offset_points).any(axis=1)].astype(NP_FLOAT_DTYPE) if offset_points is not None else None,
            "buildings": buildings_data,  # Теперь содержит больше информации о зданиях
            "metadata": {
                "polygon_area": float(polygon_area),
                "floors": int(floors),
                "setback": float(stbck),
                "max_living_area": float(max_living_area),
                "max_footprint_area": float(max_footprint_area),
                "footprint_area": float(footprint_area),
                "num_buildings": len(buildings)
            }
        }
        
        np.save(filename, data, allow_pickle=True)
        
    def calculate_max_construction_area(self, plot_area, floors):
        """Вычисляет максимальную площадь застройки."""
        density = self.config.max_density_table.get(min(floors, 25), 22.2)
        plot_area_ha = plot_area / 10000
        max_living_area = density * plot_area_ha * 1000
        max_footprint_area = max_living_area / floors / 0.7
        
        return float(max_living_area), float(max_footprint_area)


class GeneratorApp:
    """Основной класс приложения для генерации участков."""
    def __init__(self):
        self.config = Config()
        self.plot_gen = PlotGenerator(self.config)
        self.file_saver = FileSaver(self.config)
        
    def generate_all_files(self):
        """Генерирует и сохраняет участки с зданиями."""
        for dir_path in self.config.output_dirs.values():
            os.makedirs(dir_path, exist_ok=True)
        
        successful = 0
        file_index = 1
        
        while successful < self.config.num_polygons:
            poly = self.plot_gen.generate_valid_polygon(self.config.skip_center_percent)
            
            variants = self.plot_gen.generate_variants(
                poly, 
                None,
                random.randint(1, self.config.max_buildings)
            )
            
            if variants:
                for variant_idx, variant in enumerate(variants, 1):
                    self._save_files(
                        poly, 
                        variant["offset_poly"], 
                        variant["buildings"], 
                        variant["floors"], 
                        variant["setback"], 
                        file_index, 
                        variant_idx
                    )
                successful += 1
                file_index += 1
        
        print(f"Успешно сохранено {successful} участков с зданиями")
    
    def _save_files(self, poly, offset_poly, buildings, floors, stbck, file_index, variant_idx):
        """Сохраняет файлы в различных форматах."""
        base_path = f"land_plot_{file_index}_variant_{variant_idx}"
        
        # DXF
        dxf_path = os.path.join(self.config.output_dirs["dxf"], f"{base_path}.dxf")
        self.file_saver.save_as_dxf(poly, offset_poly, buildings, dxf_path, floors, stbck)
        
        # JSON
        json_path = os.path.join(self.config.output_dirs["json"], f"{base_path}.json")
        self.file_saver.save_as_json(poly, offset_poly, buildings, json_path, floors, stbck)
        
        # PNG
        png_path = os.path.join(self.config.output_dirs["png"], f"{base_path}.png")
        self.file_saver.save_as_png(poly, offset_poly, buildings, png_path, floors, stbck)
        
        # NumPy
        numpy_path = os.path.join(self.config.output_dirs["numpy"], f"{base_path}.npy")
        self.file_saver.save_as_numpy(poly, offset_poly, buildings, numpy_path, floors, stbck)


if __name__ == "__main__":
    app = GeneratorApp()
    app.generate_all_files()

Успешно сохранено 10 участков с зданиями
